In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import time
import sys
import requests
from bs4 import BeautifulSoup

In [2]:
#explore another approach - split data by 24 rows and mean for each 24 rows
def get_avg_aqi():
    aqi_df = pd.DataFrame()
    for year in range(2013,2019):
        days_avg = []
        for rows in pd.read_csv('Data/AQI/aqi{}.csv'.format(year), chunksize=24): #improves performance lower memory usage
            sum = 0
            avg = 0.0
            df = pd.DataFrame(data=rows)
            pm2 = df['PM2.5']
            for i in pm2:
                if type(i) is float or type(i) is int:
                    sum = sum+i
                elif type(i) is str:
                        if i!='NoData' and i!='PwrFail' and i!='---' and i!='InVld':
                        temp = float(i)
                        sum = sum+temp
            avg = sum/24
            days_avg.append(avg)
        aqi_df['{}'.format(year)] = pd.Series(days_avg)
    return aqi_df

In [3]:
start1 = time.time()
df = get_avg_aqi()
stop1 = time.time()
print("Time taken {}".format(stop1-start1))

Time taken 23.593676567077637


In [4]:
df.head()

,2013,2014,2015,2016,2017,2018
0,284.795833,255.875000,172.750000,284.795833,284.795833,255.875000
1,219.720833,217.083333,146.791667,219.720833,219.720833,217.083333
2,182.187500,229.166667,113.875000,182.187500,182.187500,229.166667
3,115.287500,392.208333,144.750000,115.287500,115.287500,392.208333
4,0.000000,536.500000,118.458333,0.000000,0.000000,536.500000


In [5]:
def test_met_data(year):
    column_names = ['T','TM','Tm', 'H','PP','VV','V','VM']
    final_yearly_df = pd.DataFrame(columns=column_names)
    for month in range(1,13):
        file_html = open('Data/html_data/{}/{}.html'.format(year,month), 'rb')
        plain_text = file_html.read()

        soup = BeautifulSoup(plain_text,'lxml')
        df_list = []
        for table in soup.findAll('table', {'class':'medias mensuales numspan'}):
            for tbody in table:
                temp_list = []
                for tr in tbody:
                    a = tr.get_text()
                    temp_list.append(a)
                df_list.append(temp_list)
        temp_df = pd.DataFrame(df_list, columns=df_list[0])
        temp_df = temp_df[['T','TM','Tm', 'H','PP','VV','V','VM']]
        temp_df = temp_df[1:len(temp_df.index)-2]
        final_yearly_df = final_yearly_df.append(temp_df,ignore_index=True,sort=False)
    return final_yearly_df

In [15]:
final_2013 = test_met_data(2013)
final_2013['PM2.5'] = df['2013']

In [7]:
final_2014 = test_met_data(2014)
final_2014['PM2.5'] = df['2014']

In [8]:
final_2015 = test_met_data(2015)
final_2015['PM2.5'] = df['2015']

In [9]:
final_2016 = test_met_data(2016)
final_2016['PM2.5'] = df['2016']

In [63]:
final_2017 = test_met_data(2017)
final_2017['PM2.5'] = df['2017']

In [64]:
final_2018 = test_met_data(2018)
final_2018['PM2.5'] = df['2018']

In [62]:
comb_df = pd.DataFrame(columns=['T','TM','Tm', 'H','PP','VV','V','VM'])
comb_df = comb_df.append(final_2013)
comb_df = comb_df.append(final_2014)
comb_df = comb_df.append(final_2015)
comb_df = comb_df.append(final_2016)
comb_df

,T,TM,Tm,H,PP,VV,V,VM,PM2.5
0,23.4,30.3,19,59,0,6.3,4.3,5.4,284.795833
1,22.4,30.3,16.9,57,0,6.9,3.3,7.6,219.720833
2,24,31.8,16.9,51,0,6.9,2.8,5.4,182.187500
6,22.8,30.2,17.6,55,0,7.7,3.7,7.6,154.037500
7,23.1,30.6,17.4,46,0,6.9,3.3,5.4,223.208333
...,...,...,...,...,...,...,...,...,...
354,21.8,29.4,15.6,67,0,6.9,3.5,5.4,288.416667
355,22.6,29,17.7,63,0,5.8,1.3,3.5,256.833333
359,20.6,30.5,14.7,62,0,6,2.2,3.5,169.000000
360,21.8,29,14.8,46,0,5.8,1.5,3.5,186.041667


In [65]:
final_2013.to_csv('Data/Final_Data/final_2013.csv')
final_2014.to_csv('Data/Final_Data/final_2014.csv')
final_2015.to_csv('Data/Final_Data/final_2015.csv')
final_2016.to_csv('Data/Final_Data/final_2016.csv')
final_2017.to_csv('Data/Final_Data/final_2017.csv')
final_2018.to_csv('Data/Final_Data/final_2018.csv')
comb_df.to_csv('Data/Final_Data/combined_data.csv')